# Optimize with Transformations

Here we employ `upoqa` with transformations enabled to optimize a house-made problem. Consider a simple objective function with an L1 penalty term:

$$
f(x, y) = (x + y - 1)^2 + a |xy|
$$

The optimal solutions are $x = 0, y = 1$ or $x = 1, y = 0$, yielding an optimal function value of zero. Direct optimization proves challenging due to the L1 term. A viable approach involves applying a **smoothing technique**:

$$
\widehat{f}(x, y) = (x + y - 1)^2 + a \sqrt{x^2y^2 + \tau}
$$

where $\tau \geq 0$. We initialize the optimization of $\widehat{f}$ using a relatively large $\tau$ value. The solution obtained from each optimization step serves as the starting point for the subsequent step. This process iterates until $\tau$ becomes sufficiently small, at which point we obtain the solution to the original problem.

For demonstration purposes, we optimize this problem using `upoqa` by treating both terms as element functions. The smoothed L1 term is interpreted as the composition of $(x,y) \to x^2y^2$ and the transformation $z \to a\sqrt{z + \tau}$. This functional composition can be implemented by configuring the `xform` parameter. The objective function thus becomes:

$$
\widehat{f}(x, y) = f_1(x, y) + h_2(f_2(x, y)),
$$

where 
$$
f_1(x, y) = (x + y - 1)^2, \quad f_2(x, y) = x^2y^2, \quad h_2(z) = a \sqrt{z + \tau}.
$$

The problem is then defined using the following code:

In [ ]:
import matplotlib.pyplot as plt
from upoqa.problems import PSProblem
import numpy as np

a = 5.0
start_tau = 1e-2
tau = start_tau

def f1(x):
    return (x[0] + x[1] - 1) ** 2

def f2(x):
    return (x[0] * x[1]) ** 2

def h2(z, tau):
    return a * np.sqrt(z + tau)

def h2_grad(z, tau):
    # :math:`\nabla h_2(z) = 0.5 * (z + tau) ** (- 0.5)`
    return 0.5 * a / np.sqrt(z + tau)

def h2_hess(z, tau):
    # :math:`\nabla^2 h_2(z) = - 0.25 * (z + tau) ** (- 1.5)`
    return - 0.25 * a/ np.pow(z + tau, 1.5)

def obj(x, tau):
    # The objective function with an L2 smoothing term (parameterized by tau)
    return f1(x) + h2(f2(x), tau) 

# Start from a far point
x0 = [-5, -5]

# The minimum function value
fopt = 0.0

# Define the partially separable structure
fun =    {'principal': f1,     'penalty': f2}
coords = {'principal': [0, 1], 'penalty': [0, 1]}

# The function, gradient and hession of the transformation
xforms = {
    'penalty': [lambda x: h2(x, tau), lambda x: h2_grad(x, tau), lambda x: h2_hess(x, tau)],
}

# Bound check for the input of transformation
xform_bounds = {'penalty': (0, np.inf)}

print("f(x0) =", obj(x0, tau))

f(x0) = 246.00099999600002


We then proceed to optimize the problem. 

A key technique employed here involves **passing a custom callback function** `callback_func` that UPOQA invokes at each iteration. This callback dynamically reduces the $\tau$ value in a stepwise manner. When the reduction magnitude is appropriately controlled, this approach enables us to obtain the solution directly within a single optimization run, bypassing the need for multiple algorithm executions.

The callback function accepts an `intermediate_result` parameter (of type `upoqa.utils.OptimizeResult`), which contains the current iteration's results. Through this object, we can access:
- The current iterate `intermediate_result.x`
- The current objective function value `intermediate_result.fun`
- And some other things, see the docstring of `upoqa.utils.OptimizeResult` for details.

To enable deeper control, we configure `upoqa` with `return_internals=True`. This grants access to the algorithm's internal state, particularly the algorithm manager (`intermediate_result.manager`). Then we can invoke `intermediate_result.manager.update_xforms()` to implement the dynamic $\tau$ updates described above.

In [ ]:
def callback_func(intermediate_result):
    # calculate the new tau
    tau = start_tau * 0.9 ** intermediate_result.nit
    # update the transformation with the new tau
    intermediate_result.manager.update_xforms({
        'penalty': [lambda x: h2(x, tau), lambda x: h2_grad(x, tau), lambda x: h2_hess(x, tau)],
    })
    
# optimize the problem with upoqa
import upoqa
res = upoqa.minimize(
    fun,
    x0=x0,
    coords=coords,
    xforms=xforms,
    xform_bounds=xform_bounds,
    callback=callback_func,
    disp=0,
    return_internals=True
)
print("Function value obtained by UPOQA:", obj(res.x, 0.0))

# direct optimization fails
import scipy
sp_res = scipy.optimize.minimize(lambda x: obj(x, tau = 0.0), x0)
print("Function value obtained by BFGS via direct optimization:", obj(sp_res.x, 0.0))

Function value obtained by UPOQA: 4.522866417616116e-10
Function value obtained by BFGS via direct optimization: 0.5555555555555798
